<a href="https://colab.research.google.com/github/chengwen-zheng/openAi-lab/blob/main/LangChain_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatGPT不会算算术


In [ ]:
%pip install langchain
%pip install openai
%pip install tiktoken
%pip install faiss-cpu


!python -m spacy download zh_core_web_sm


from googleapiclient.discovery import build
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import io
import json
import os
import openai

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()

# Build the Drive API client
drive_service = build('drive', 'v3')
file_id = '14x1k7ErttIPXRcKN53g_Ogn3zfn6AEM_'
file = drive_service.files().get(fileId=file_id).execute()
content = drive_service.files().get_media(fileId=file_id).execute()

# Read the env config.json. set env for OPENAI_API_KEY
with io.BytesIO(content) as f:
    config = json.load(f)
os.environ["OPENAI_API_KEY"] = config["api_key"];
openai.api_key = os.environ["OPENAI_API_KEY"] ;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import openai, os

openai.api_key = os.environ.get("OPENAI_API_KEY")

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain


llm = OpenAI(model_name="text-davinci-003", max_tokens=2048, temperature=0.5)
multiply_prompt = PromptTemplate(template="请计算一下{question}是多少?", input_variables=["question"])
math_chain = LLMChain(llm=llm, prompt=multiply_prompt, output_key="answer")
answer = math_chain.run({"question": "352乘以493"})
print("OpenAI API 说答案是:", answer)

python_answer = 352 * 493
print("Python 说答案是:", python_answer)

OpenAI API 说答案是: 

352 x 493 = 173936
Python 说答案是: 173536


In [ ]:
multiply_by_python_prompt = PromptTemplate(template="请写一段Python代码，计算{question}?", input_variables=["question"])
math_chain = LLMChain(llm=llm, prompt=multiply_by_python_prompt, output_key="answer")
answer = math_chain.run({"question": "352乘以493"})
print(answer)



print(352 * 493)


In [ ]:
multiply_by_python_prompt = PromptTemplate(template="请写一段Python代码，计算{question}?", input_variables=["question"])
math_chain = LLMChain(llm=llm, prompt=multiply_by_python_prompt, output_key="answer")
answer_code = math_chain.run({"question": "352乘以493"})

from langchain.utilities import PythonREPL
python_repl = PythonREPL()
result = python_repl.run(answer_code)
print(result)

173536



# 通过LLMMathChain算算术


In [ ]:
from langchain import LLMMathChain

llm_math = LLMMathChain(llm=llm, verbose=True)
result = llm_math.run("请计算一下352乘以493是多少?")
print(result)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm_math/base.py:50: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new LLMMathChain chain...
请计算一下352乘以493是多少?
```text
352 * 493
```
...numexpr.evaluate("352 * 493")...

Answer: 173536
> Finished chain.
Answer: 173536


# 通过Google搜索查天气

In [ ]:
from langchain.chains import LLMRequestsChain

template = """在 >>> 和 <<< 直接是来自Google的原始搜索结果.
请把对于问题 '{query}' 的答案从里面提取出来，如果里面没有相关信息的话就说 "找不到"
请使用如下格式：
Extracted:<answer or "找不到">
>>> {requests_result} <<<
Extracted:"""

PROMPT = PromptTemplate(
    input_variables=["query", "requests_result"],
    template=template,
)
requests_chain = LLMRequestsChain(llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=PROMPT))
question = "怎么样购买腾讯股票？"
inputs = {
    "query": question,
    "url": "https://www.google.com/search?q=" + question.replace(" ", "+")
}
result=requests_chain(inputs)
print(result)
print(result['output'])

{'query': '怎么样购买腾讯股票？', 'url': 'https://www.google.com/search?q=怎么样购买腾讯股票？', 'output': '首先腾讯控股是港股。 有两种方式可以买到。 第一在你的股票账户里开通港股通，但是有门槛要求，要连续20个交易日以上持有50万市值才可以开通。 第二：直接开通港股。'}
首先腾讯控股是港股。 有两种方式可以买到。 第一在你的股票账户里开通港股通，但是有门槛要求，要连续20个交易日以上持有50万市值才可以开通。 第二：直接开通港股。


# 通过TransformChain运行自定义函数

In [ ]:
import re

def parse_weather_info(weather_info)->dict:
    # 将天气信息拆分成不同部分
    parts = weather_info.split('; ')
    print(parts)
    # 解析天气
    weather = parts[0].strip()

    # 解析温度范围，并提取最小和最大温度
    temperature_range = parts[1].strip().replace('℃', '').split('～')
    print(temperature_range)
    temperature_min = int(temperature_range[0])
    temperature_max = int(temperature_range[1])

    # 解析风向和风力
    wind_parts = parts[2].split(' ')
    wind_direction = wind_parts[0].strip()
    wind_force = wind_parts[1].strip()
    weather_dict = {
        'weather': weather,
        'temperature': {
            'min': temperature_min,
            'max': temperature_max,
        },
        'wind': {
            'direction': wind_direction,
            'level': wind_force
        }
    }
    return weather_dict

# today the weather is changeed, so the format is incorrect. so params of weather is fixed.
weather_dict = parse_weather_info("小雨; 10℃～15℃; 东北风 风力4-5级")
print(weather_dict)

['小雨', '10℃～15℃', '东北风 风力4-5级']
['10', '15']
{'weather': '小雨', 'temperature': {'min': 10, 'max': 15}, 'wind': {'direction': '东北风', 'level': '风力4-5级'}}


In [ ]:
from langchain.chains import TransformChain, SequentialChain

def transform_func(inputs: dict) -> dict:
    
    text ="小雨; 10℃～15℃; 东北风 风力4-5级"
    return {"weather_info" : parse_weather_info(text)}

transformation_chain = TransformChain(input_variables=["output"], 
                                      output_variables=["weather_info"], transform=transform_func)

final_chain = SequentialChain(chains=[requests_chain, transformation_chain], 
                              input_variables=["query", "url"], output_variables=["weather_info"])
final_result = final_chain.run(inputs)

['小雨', '10℃～15℃', '东北风 风力4-5级']
['10', '15']


# 通过VectorDBQA做问答

In [ ]:
page_content = '''Q: 如何注册新帐户？
A: 点击网站右上角的“注册”按钮，然后按照提示填写相关信息并设置密码。完成后，您将收到一封验证电子邮件。点击邮件中的链接以激活您的帐户。

Q: 忘记密码怎么办？
A: 点击登录页面的“忘记密码”链接，输入您的电子邮件地址。我们将向您发送一封包含重置密码链接的邮件。请点击链接并按照提示操作。

Q: 如何更改帐户信息？
A: 登录您的帐户，然后点击“我的帐户”以更改个人信息、收货地址等。

Q: 如何下订单？
A: 浏览商品并将想要购买的商品加入购物车。当您准备结算时，点击购物车图标，然后选择送货地址、付款方式和物流方式。确认订单详情后，点击“提交订单”。

Q: 支持哪些支付方式？
A: 我们支持支付宝、微信支付、银联在线支付、信用卡支付等多种支付方式。

Q: 如何查看订单状态？
A: 登录您的帐户，点击“我的订单”，在此页面上，您可以查看所有订单及其当前状态。

Q: 如何取消订单？
A: 在订单发货前，您可以登录帐户，进入“我的订单”页面取消订单。如果订单已发货，您需要联系客服协助处理。

Q: 如何申请退款？
A: 登录帐户，进入“我的订单”页面，选择要退款的订单并点击“申请退款”。请按照页面提示填写退款理由和退款金额。提交申请后，我们将在1-3个工作日内处理您的请求。

Q: 退货政策是什么？
A: 自收到商品之日起7天内，如产品未使用、包装完好，您可以申请退货。某些特殊商品可能不支持退货，请在购买前查看商品详情页面的退货政策。

Q: 退款会退回到哪里？
A: 退款将原路退回至您的支付帐户。

Q: 支持哪些省份配送？
A: 我们支持全国大部分省份的配送，包括北京、上海、天津、重庆、河北、山西、辽宁、吉林、黑龙江、江苏、浙江、安徽、福建、江西、山东、河南、湖北、湖南、广东、海南、四川、贵州、云南、陕西、甘肃、青海、台湾、内蒙古、广西、西藏、宁夏和新疆

Q: 物流时效是多久？
A: 一般情况下，大部分城市的订单在2-3个工作日内送达，偏远地区可能需要5-7个工作日。具体送货时间可能因订单商品、配送地址和物流公司而异。

Q: 如何查询物流信息？
A: 在订单发货后，您可以在“我的订单”页面查看物流单号。点击相应订单的“查看物流”按钮，输入物流单号查询物流详情。

Q: 提供哪些快递公司的服务？
A: 我们与顺丰速运、圆通速递、申通快递、韵达快递、中通快递、百世快递等多家知名快递公司合作。

Q: 如何更改收货地址？
A: 在订单发货前，您可以登录帐户，进入“我的订单”页面，选择要修改的订单并点击“修改地址”。如果订单已发货，您需要联系客服协助处理。

Q: 如何查询发票信息？
A: 登录您的帐户，进入“我的发票”页面。在此页面上，您可以查看已开具的发票及其详细信息。

Q: 如何申请开具发票？
A: 在提交订单时，您可以选择需要发票，并填写相关发票信息。您还可以在订单完成后登录帐户，进入“我的发票”页面，点击“申请发票”按钮并填写发票信息。

Q: 提供哪些类型的发票？
A: 我们提供普通发票和增值税专用发票。具体发票类型请参考订单页面或联系客服咨询。

Q: 为什么我的订单被取消？
A: 订单可能因库存不足、支付异常、用户要求等原因被取消。如有疑问，请联系客服咨询。

Q: 如何参加促销活动？
A: 关注我们的官方网站和社交媒体账号，我们会定期发布优惠券、折扣码和特价商品信息。您还可以在购物车页面直接查看适用于您订单的优惠活动。

Q: 如何使用优惠券？
A: 在购物车页面，输入优惠券代码后，点击“应用”。优惠券折扣将自动应用于您的订单。

Q: 优惠券有使用限制吗？
A: 优惠券可能有最低消费要求、有效期限制或特定商品范围。使用前请查看优惠券详情。'''

In [ ]:

import zh_core_web_sm
nlp = zh_core_web_sm.load()
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import SpacyTextSplitter
from langchain import OpenAI, VectorDBQA

llm = OpenAI(temperature=0)
text_splitter = SpacyTextSplitter(chunk_size=256, pipeline="zh_core_web_sm")
texts = text_splitter.create_documents(texts=page_content)

embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_documents(texts, embeddings)

faq_chain = VectorDBQA.from_chain_type(llm=llm, vectorstore=docsearch, verbose=True)

In [ ]:
question = "请问你们的货，能送到三亚吗？大概需要几天？"
result = faq_chain.run(question)
print(result)



> Entering new VectorDBQA chain...

> Finished chain.
 是的，我们的货可以送到三亚。大概需要3-5天。


In [ ]:
question = "请问你们的退货政策是怎么样的？" 
result = faq_chain.run(question)
print(result)



> Entering new VectorDBQA chain...

> Finished chain.
 我不知道，但是您可以通过电子邮件联系我们的客服团队来了解更多信息。
